In [2]:
from google.colab import files
uploaded = files.upload()

Saving raw_data.parquet to raw_data.parquet


In [4]:
import pandas as pd

# Assuming the file name is 'your_file.parquet'
df = pd.read_parquet('raw_data.parquet')


In [5]:
print(df)

       latitude  longitude             timestamp  unit
0      40.23426  -77.13796  2023-12-05T23:13:40Z  1000
1      40.23422  -77.13796  2023-12-05T23:13:40Z  1000
2      40.23419  -77.13795  2023-12-05T23:13:40Z  1000
3      40.23412  -77.13792  2023-12-05T23:13:40Z  1000
4      40.23412  -77.13792  2023-12-05T23:13:40Z  1000
...         ...        ...                   ...   ...
73467  42.98914  -74.40199  2023-12-11T02:50:02Z  2000
73468  42.98919  -74.40328  2023-12-11T02:50:09Z  2000
73469  42.98919  -74.40328  2023-12-11T02:50:09Z  2000
73470  42.98919  -74.40328  2023-12-11T02:50:09Z  2000
73471  42.98919  -74.40332  2023-12-11T02:50:09Z  2000

[73472 rows x 4 columns]


In [18]:
import pandas as pd
from datetime import datetime, timedelta

import pandas as pd
from datetime import timedelta

# Assuming 'df' is your DataFrame
# Convert 'timestamp' column to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort the DataFrame by unit and timestamp
df.sort_values(['unit', 'timestamp'], inplace=True)

# Function to determine trip number based on time difference
def get_trip_number(timestamps):
    trip_number = 0
    trip_start_time = timestamps.iloc[0]

    for timestamp in timestamps:
        if timestamp - trip_start_time > timedelta(hours=7):
            trip_start_time = timestamp
            trip_number += 1

    return trip_number
df['trip_number'] = df.groupby('unit')['timestamp'].transform(get_trip_number)

for (unit, trip), trip_data in df.groupby(['unit', 'trip_number']):
    trip_data.to_csv(f'{unit}_{trip}.csv', index=False)




In [20]:
!unzip MapUp-Data-Assessment-E-main.zip

Archive:  MapUp-Data-Assessment-E-main.zip
4abf9fc911cabf5b8751847a79708cf5ce2fc5fc
   creating: MapUp-Data-Assessment-E-main/
  inflating: MapUp-Data-Assessment-E-main/.env  
 extracting: MapUp-Data-Assessment-E-main/.gitignore  
  inflating: MapUp-Data-Assessment-E-main/README.md  
   creating: MapUp-Data-Assessment-E-main/evaluation_data/
   creating: MapUp-Data-Assessment-E-main/evaluation_data/input/
  inflating: MapUp-Data-Assessment-E-main/evaluation_data/input/raw_data.parquet  
  inflating: MapUp-Data-Assessment-E-main/excel-assessment.xlsm  
  inflating: MapUp-Data-Assessment-E-main/requirements.txt  
   creating: MapUp-Data-Assessment-E-main/sample_data/
   creating: MapUp-Data-Assessment-E-main/sample_data/input/
  inflating: MapUp-Data-Assessment-E-main/sample_data/input/test_data.parquet  
   creating: MapUp-Data-Assessment-E-main/sample_data/output/
   creating: MapUp-Data-Assessment-E-main/sample_data/output/process1/
  inflating: MapUp-Data-Assessment-E-main/sample_dat

In [16]:
%run process1.py --to_process /path/to/your/input_file.parquet --output_dir /path/to/your/output_directory





File not found: C:\Users\VERSHA\Desktop\project\1000_0.csv


In [21]:
pip install requests python-dotenv


In [28]:
with open('.env', 'w') as f:
    f.write("TOLLGURU_API_KEY=your_api_key\n")
    f.write("TOLLGURU_API_URL=https://apis.tollguru.com\n")


In [29]:
from dotenv import load_dotenv
load_dotenv()


True

In [30]:
import os
print(os.getenv('TOLLGURU_API_KEY'))
print(os.getenv('TOLLGURU_API_URL'))


your_api_key
https://apis.tollguru.com


In [31]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# TollGuru API key and URL
TOLLGURU_API_KEY = os.getenv('TOLLGURU_API_KEY')
TOLLGURU_API_URL = os.getenv('TOLLGURU_API_URL')

# API parameters
API_PARAMETERS = {'vehicleType': '5AxlesTruck', 'mapProvider': 'osrm'}

# Function to upload a CSV file to TollGuru API
def upload_to_tollguru(file_path):
    if not TOLLGURU_API_URL:
        raise ValueError("TOLLGURU_API_URL is not defined in the environment variables.")

    url = f'{TOLLGURU_API_URL}/toll/v2/gps-tracks-csv-upload'
    headers = {'x-api-key': TOLLGURU_API_KEY, 'Content-Type': 'text/csv'}

    with open(file_path, 'rb') as file:
        response = requests.post(url, data=file, headers=headers)

    return response.json()

def main():
    # Assuming you have a list of CSV files obtained from Process1
    csv_files = [
        '/content/MapUp-Data-Assessment-E-main/sample_data/output/process1/2000_0.csv',
        '/content/MapUp-Data-Assessment-E-main/sample_data/output/process1/2000_1.csv',  # Add more file paths as needed
        # ...
    ]

    # Create a ThreadPoolExecutor for concurrent uploads
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Concurrently upload CSV files to TollGuru API
        responses = list(executor.map(upload_to_tollguru, csv_files))

    # Save JSON responses to files with the same names
    for file_path, response in zip(csv_files, responses):
        json_file_path = f'{os.path.splitext(file_path)[0]}.json'
        with open(json_file_path, 'w') as json_file:
            json_file.write(str(response))

if __name__ == "__main__":
    main()


In [33]:
import os
import json
import csv

def process_json_file(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)

    # Extract relevant data from the JSON file
    unit = data.get('unit', '')
    trip_id = os.path.splitext(os.path.basename(json_file))[0]
    toll_data = data.get('tollData', [])

    # Transform data into CSV format
    csv_data = []
    for toll_info in toll_data:
        row = {
            'unit': unit,
            'trip_id': trip_id,
            'toll_loc_id_start': toll_info.get('startToll', {}).get('tollID', ''),
            'toll_loc_id_end': toll_info.get('endToll', {}).get('tollID', ''),
            'toll_loc_name_start': toll_info.get('startToll', {}).get('tollName', ''),
            'toll_loc_name_end': toll_info.get('endToll', {}).get('tollName', ''),
            'toll_system_type': toll_info.get('tollSystemType', ''),
            'entry_time': toll_info.get('entryTime', ''),
            'exit_time': toll_info.get('exitTime', ''),
            'tag_cost': toll_info.get('tagCost', ''),
            'cash_cost': toll_info.get('cashCost', ''),
            'license_plate_cost': toll_info.get('licensePlateCost', ''),
        }
        csv_data.append(row)

    return csv_data

def process_json_files(input_dir, output_file):
    csv_header = [
        'unit', 'trip_id', 'toll_loc_id_start', 'toll_loc_id_end',
        'toll_loc_name_start', 'toll_loc_name_end', 'toll_system_type',
        'entry_time', 'exit_time', 'tag_cost', 'cash_cost', 'license_plate_cost'
    ]

    with open(output_file, 'w', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=csv_header)
        writer.writeheader()

        for root, _, files in os.walk(input_dir):
            for file in files:
                if file.endswith('.json'):
                    json_file_path = os.path.join(root, file)
                    csv_data = process_json_file(json_file_path)
                    writer.writerows(csv_data)

if __name__ == "__main__":
    input_directory = '/content/MapUp-Data-Assessment-E-main/sample_data/output/process2/'
    output_csv_file = '/content/MapUp-Data-Assessment-E-main/sample_data/output/process3/transformed_data.csv'

    process_json_files(input_directory, output_csv_file)


In [36]:
!zip -r Full .

  adding: .config/ (stored 0%)
  adding: .config/configurations/ (stored 0%)
  adding: .config/configurations/config_default (deflated 15%)
  adding: .config/logs/ (stored 0%)
  adding: .config/logs/2023.12.04/ (stored 0%)
  adding: .config/logs/2023.12.04/14.20.49.627769.log (deflated 91%)
  adding: .config/logs/2023.12.04/14.23.49.652015.log (deflated 58%)
  adding: .config/logs/2023.12.04/14.23.59.638040.log (deflated 86%)
  adding: .config/logs/2023.12.04/14.26.59.279282.log (deflated 57%)
  adding: .config/logs/2023.12.04/14.27.00.107426.log (deflated 56%)
  adding: .config/logs/2023.12.04/14.26.48.840108.log (deflated 58%)
  adding: .config/active_config (stored 0%)
  adding: .config/config_sentinel (stored 0%)
  adding: .config/.last_opt_in_prompt.yaml (stored 0%)
  adding: .config/default_configs.db (deflated 98%)
  adding: .config/.last_survey_prompt.yaml (stored 0%)
  adding: .config/.last_update_check.json (deflated 22%)
  adding: .config/gce (stored 0%)
  adding: Full/ (sto